In [214]:
import random
import math
from typing import Callable

In [215]:
def even_int_gen(a: int, b: int) -> int:
    while True:
        yield round(a + random.random()*(b-a))


def exp_int_gen(l: float) -> int:
    while True:
        yield (-1) * math.log(random.random()) / l


# тут аннотации неоч верные скорее всего, ибо yield требует SupprotsNext, а я пишу Callable
def imit_model(items_count: int, stands_count: int,
               release_gen: Callable[[int, int], int],
               check_gen: Callable[[int, int], int]) -> float:

    stands = [0 for _ in range(stands_count)]
    not_checked = 0

    for i in range(items_count):
        rel_time = next(release_gen)

        for j in range(len(stands)):
            stands[j] = max(0, stands[j] - rel_time)

        found_free = False
        for j in range(len(stands)):
            if stands[j] == 0:
                stands[j] = next(check_gen)
                found_free = True
                break

        if not found_free:
            not_checked += 1

    return (items_count - not_checked) / items_count


In [228]:
# a, b = 5, 15
# c, d = 15, 25
# release_gen = even_int_gen(a, b)
# check_gen = even_int_gen(c, d)


l = 0.95
u = 1
release_gen = exp_int_gen(l)
check_gen = exp_int_gen(u)


def p_priema_by_erlang(l: float, u: float, stands_count: int) -> float:
    alpha = l / u
    n = stands_count
    up = alpha**n / math.factorial(n)
    down = 1
    for i in range (1, n + 1):
        down += alpha**i / math.factorial(i)
    return 1 - up / down


need = 0.98
items_count = 10**5
stands_count = 1

while True:
    result = imit_model(items_count, stands_count, release_gen, check_gen)
    erl_result = p_priema_by_erlang(l, u, stands_count)
    print(stands_count, result, erl_result)
    stands_count += 1
    if result >= need:
        break

# 1 0.5106
# 2 0.8158
# 3 0.9391
# 4 0.9891


1 0.51336 0.5128205128205128
2 0.81283 0.8120770432066632
3 0.94248 0.9438334739065993
4 0.98704 0.9868360512052669
